In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics==8.0.20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 30.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [5]:
import zipfile
import ultralytics
from ultralytics import YOLO

from IPython import display
from IPython.display import display, Image
from ultralytics import YOLO
from IPython.display import display, Image

zip_file_path = "/content/drive/MyDrive/drive-download-20240713T173959Z-001"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

display.clear_output()

ultralytics.checks()
Image(filename='/content/runs/detect/train/confusion_matrix.png', width=600)
Image(filename='/content/runs/detect/train/results.png', width=600)

!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=data.yaml
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.25 source=test/images

IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/drive-download-20240713T173959Z-001'

In [ ]:
import os
import zipfile
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras import Input
import numpy as np


zip_file_path = "/content/drive/MyDrive/homer_bart.zip"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

image_dir="/content/homer_bart"

# Create a dataset from the directory
#dataset = tf.keras.preprocessing.image_dataset_from_directory(
#    image_dir,
 #   label_mode="binary",
 #   image_size=(64, 64),
  #  batch_size=32,
#)

def load_images(directory, target_size=(64, 64)):
    images = []
    labels = []
    label_encoder = LabelEncoder()

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith((".jpg", ".png", ".bmp")):
                img_path = os.path.join(root, file)
                label = os.path.basename(root)
                labels.append(label)
                img = Image.open(img_path)
                img = img.resize(target_size)
                img = np.array(img) / 255.0
                images.append(img)

    labels = label_encoder.fit_transform(labels)

    return np.array(images), np.array(labels)

images, labels = load_images(images_directory)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)

model = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=[BinaryAccuracy])

model.fit(X_train, y_train, batch_size=32, epochs=64)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
